# Aicon 1

## Action Making

In [1]:
# Chunk 1: Imports
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime

# Add parent directory to path for imports
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
sys.path.append(str(project_root))

# Import the required classes
from aicons.definitions.aicon import AIcon
from aicons.bayesbrainGPT.sensors.meta_s.meta_ads_sales_sensor import MetaAdsSalesSensor

In [2]:
# Chunk 2: Create AIcon
aicon = AIcon("marketing_aicon")
print(f"AIcon created: {aicon.name}")

AIcon created: marketing_aicon


In [3]:
aicon.add_state_factor(
    name="purchases",
    factor_type="continuous",
    value=0.0,
    params={
        "loc": 0.0,
        "scale": 1.0,
        "constraints": {"lower": 0}
    },
    relationships={
        "depends_on": []  # Empty list for root factor
    }
)

aicon.add_state_factor(
    name="add_to_carts",
    factor_type="continuous",
    value=0.0,
    params={
        "loc": 0.0,
        "scale": 5.0,
        "constraints": {"lower": 0}
    },
    relationships={
        "depends_on": []  # Empty list for root factor
    }
)

aicon.add_state_factor(
    name="initiated_checkouts",
    factor_type="continuous",
    value=0.0,
    params={
        "loc": 0.0,
        "scale": 2.0,
        "constraints": {"lower": 0}
    },
    relationships={
        "depends_on": []  # Empty list for root factor
    }
)

ContinuousLatentVariable(name='initiated_checkouts', value=0.0, description='')

In [4]:
# Chunk 3: Setup Meta Ads sensor
access_token = "EAAZAn8wmq1IEBOZCz8oyDZBBgiazAgnQKIoAr4mFTbkV7jxi6t3APzOSxFybXNIkBgwQACdagbs5lFE8tpnNOBOOpWtS3KjZAdf9MNAlySpwEaDrX32oQwUTNmOZAaSXjT5Os5Q8YqRo57tXOUukB7QtcO8nQ8JuqrnnshCr7A0giynZBnJKfuPakrZBWoZD"
ad_account_id = "act_252267674525035"
campaign_id = "120218631288730217"

sensor = MetaAdsSalesSensor(
    name="meta_ads",
    reliability=0.9,
    access_token=access_token,
    ad_account_id=ad_account_id,
    campaign_id=campaign_id,
    api_version="v18.0",
    time_granularity="hour"
)

aicon.add_sensor("meta_ads", sensor)
print("Meta Ads sensor added")

✅ Successfully initialized Meta Ads API vv18.0 for account act_252267674525035
Found 5 ad sets in campaign 120218631288730217
Found 14 ads in 5 ad sets
Found 3 active ads out of 14 total ads
Found 3 active ads
- Ad ID: 120219379327910217, Name: Anuncio 02 - Marzo - Foto - GABITO
- Ad ID: 120219835173950217, Name: Anuncio 9 - Nata + Gabito
- Ad ID: 120218905466570217, Name: Anuncio 2 - Carrusel 1 (3 artistas)
Number of factors: 21
Factor names: purchases, add_to_carts, initiated_checkouts, ad_120219379327910217_purchases, ad_120219379327910217_add_to_carts, ad_120219379327910217_initiated_checkouts, ad_120219379327910217_impressions, ad_120219379327910217_clicks, ad_120219379327910217_spend, ad_120219835173950217_purchases...
Auto-creating missing factor: ad_120219379327910217_purchases (continuous)
Auto-creating missing factor: ad_120219379327910217_add_to_carts (continuous)
Auto-creating missing factor: ad_120219379327910217_initiated_checkouts (continuous)
Auto-creating missing facto

In [5]:
# Chunk 4: Define action space
aicon.define_action_space(
    space_type='marketing',
    total_budget=1000.0,
    num_ads=2,
    budget_step=100.0,
    ad_names=['google', 'facebook']
)

ActionSpace(dimensions=2, constraints=1)

In [6]:
print(aicon.brain.action_space.raw_print())

dimensions: ['google_budget: [0.0, 1000.0] step=100.0', 'facebook_budget: [0.0, 1000.0] step=100.0']
size: 121 (stepped continuous)
constraints: ['<function create_marketing_ads_space.<locals>.budget_sum_constraint at 0x17b8f0c20>']
actions: [{'google_budget': np.float64(0.0), 'facebook_budget': np.float64(1000.0)}, {'google_budget': np.float64(100.0), 'facebook_budget': np.float64(900.0)}, {'google_budget': np.float64(200.0), 'facebook_budget': np.float64(800.0)}, {'google_budget': np.float64(300.0), 'facebook_budget': np.float64(700.0)}, {'google_budget': np.float64(400.0), 'facebook_budget': np.float64(600.0)}, {'google_budget': np.float64(500.0), 'facebook_budget': np.float64(500.0)}, {'google_budget': np.float64(600.0), 'facebook_budget': np.float64(400.0)}, {'google_budget': np.float64(700.0), 'facebook_budget': np.float64(300.0)}, {'google_budget': np.float64(800.0), 'facebook_budget': np.float64(200.0)}, {'google_budget': np.float64(900.0), 'facebook_budget': np.float64(100.0)}

In [7]:
# Chunk 5: Define utility function
utility = aicon.define_utility_function(
    utility_type='linear',
    name="Marketing Budget Utility",
    weights={'google': 0.6, 'facebook': 0.4}
)


Utility Function: LinearUtility: Σ(weights * values) where weights = [tf.Tensor(0.6, shape=(), dtype=float32), tf.Tensor(0.4, shape=(), dtype=float32)]


In [8]:
# Chunk 6: Update from Meta Ads sensor
print("Updating beliefs from Meta Ads sensor...")
aicon.update_from_sensor("meta_ads")

Updating beliefs from Meta Ads sensor...

=== Starting Belief Update ===
Current time: 2025-03-30 12:09:00
No posterior samples available
Campaign objective: OUTCOME_SALES
Found 5 ad sets in campaign 120218631288730217
Found 14 ads in 5 ad sets
Trying date_preset: today
Fetching data for 14 ads with today preset...
Retrieved 39 records
Found latest hour: 11:00:00 - 11:59:59
Filtered to 3 records for hour 11:00:00 - 11:59:59
Found 3 hourly records
Extracted factors for 3 ads
Example ad IDs: 120218905466570217, 120219379327910217, 120219835173950217
Example factor names: purchases, add_to_carts, initiated_checkouts, ad_120218905466570217_purchases, ad_120218905466570217_add_to_carts, ad_120218905466570217_initiated_checkouts, ad_120218905466570217_impressions, ad_120218905466570217_clicks, ad_120218905466570217_spend, ad_120219379327910217_purchases
Mapping observation: purchases → purchases
Mapping observation: add_to_carts → add_to_carts
Mapping observation: initiated_checkouts → initi

/Users/infa/Library/Caches/pypoetry/virtualenvs/babel-PAVoc7qP-py3.12/lib/python3.12/site-packages/tensorflow_probability/python/__init__.py:70: UserWarning: TensorFloat-32 matmul/conv are enabled for NVIDIA Ampere+ GPUs. The resulting loss of precision may hinder MCMC convergence. To turn off, run `tf.config.experimental.enable_tensor_float_32_execution(False)`. For more detail, see https://github.com/tensorflow/community/pull/287.
  warnings.warn(



Updating state factors from posterior:
  purchases: 0.0000 -> -0.0134 (std: 0.7239)
  add_to_carts: 0.0000 -> -0.0262 (std: 1.0974)
  initiated_checkouts: 0.0000 -> 0.7115 (std: 1.0176)
  ad_120219379327910217_purchases: 0.0000 -> 0.0270 (std: 0.7112)
  ad_120219379327910217_add_to_carts: 0.0000 -> 0.0277 (std: 1.0896)
  ad_120219379327910217_initiated_checkouts: 0.0000 -> 0.7251 (std: 0.9374)
  ad_120219379327910217_impressions: 0.0000 -> 1179.8304 (std: 1.1141)
  ad_120219379327910217_clicks: 0.0000 -> 25.7355 (std: 1.0784)
  ad_120219379327910217_spend: 0.0000 -> 23.3553 (std: 1.1201)
  ad_120219835173950217_purchases: 0.0000 -> -0.0072 (std: 0.7221)
  ad_120219835173950217_add_to_carts: 0.0000 -> 0.0089 (std: 1.0941)
  ad_120219835173950217_initiated_checkouts: 0.0000 -> 0.0615 (std: 0.9159)
  ad_120219835173950217_impressions: 0.0000 -> 10361.7031 (std: 1.1119)
  ad_120219835173950217_clicks: 0.0000 -> 351.6285 (std: 1.1205)
  ad_120219835173950217_spend: 0.0000 -> 130.7805 (std:

True

In [9]:
best_action, expected_utility = aicon.find_best_action(num_samples=100)


=== Starting find_best_action ===
Action space type: <class 'aicons.bayesbrainGPT.decision_making.action_space.ActionSpace'>
Utility function type: <class 'aicons.bayesbrainGPT.utility_function.linear_utility.LinearUtility'>

Getting posterior samples from perception system...

Posterior samples shape: 1000 samples
Posterior sample keys: ['purchases', 'add_to_carts', 'initiated_checkouts', 'ad_120219379327910217_purchases', 'ad_120219379327910217_add_to_carts', 'ad_120219379327910217_initiated_checkouts', 'ad_120219379327910217_impressions', 'ad_120219379327910217_clicks', 'ad_120219379327910217_spend', 'ad_120219835173950217_purchases', 'ad_120219835173950217_add_to_carts', 'ad_120219835173950217_initiated_checkouts', 'ad_120219835173950217_impressions', 'ad_120219835173950217_clicks', 'ad_120219835173950217_spend', 'ad_120218905466570217_purchases', 'ad_120218905466570217_add_to_carts', 'ad_120218905466570217_initiated_checkouts', 'ad_120218905466570217_impressions', 'ad_12021890546

-------------------